generator : U-net

discriminator : patchGAN

no gp, no sn, no sa;

# INSTALL TENSORFLOW

In [0]:
# Tensorboard
! pip install --upgrade tensorflow-gpu 
! pip uninstall tensorflow-tensorboard
! pip install tensorboard

# To generate GIFs
!pip install -q imageio

# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
print(tf.__version__)
print(str(tf.test.is_gpu_available()))

     |████████████████████████████████| 380.8MB 39kB/s 
     |████████████████████████████████| 450kB 40.7MB/s 
     |████████████████████████████████| 3.8MB 25.1MB/s 
     |████████████████████████████████| 81kB 10.2MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.1 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled goog

     |████████████████████████████████| 2.4MB 2.8MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
  Found existing installation: grpcio 1.15.0
    Uninstalling grpcio-1.15.0:
      Successfully uninstalled grpcio-1.15.0


2.0.0
True


In [0]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/'My Drive'/GAN/cycleGAN/
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
/content/drive/My Drive/GAN/cycleGAN
ckpt  data  ipynb  logs  mask2img  mask2img_1  p10  s10  test  train  v10


In [0]:
# %tensorboard --logdir ./logs

# IMPORT

In [0]:
import os
import PIL
import time
import glob
import imageio
import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import metrics
from IPython import display

# For FID
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from skimage.transform import resize


inception3 = tf.keras.applications.InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

print(tf.__version__)
print(str(tf.test.is_gpu_available()))
!ls


87916544/87910968 [==============================] - 1s 0us/step
2.0.0
True
ckpt  data  ipynb  logs  mask2img  mask2img_1  p10  s10  test  train  v10


In [0]:
HEIGHT = 300
WIDTH = 300
IMG_HEIGHT = 256
IMG_WIDTH = 256
DATA = './data/'
glamda = 3
dlamda = 0
batch_size = 64
begin = 0
epochs = 1500
train_size = 640
test_size = 320
drop_rate = 0.3
version = 'mask2img'
JJ = 4

# DATA

In [0]:
def load_img(img_path, mask_path):
    input_image = tf.image.decode_jpeg(tf.io.read_file(img_path))
    msk_image = tf.image.decode_jpeg(tf.io.read_file(mask_path))
    input_image = tf.cast(input_image, tf.float32)
    msk_image = tf.cast(msk_image/80, tf.int32)
    msk_image = tf.image.random_crop(msk_image, [500, 500, 1])
    input_image = (input_image / 127.5) - 1
    
    return input_image, msk_image

def random_crop(img, msk):
    img = tf.image.resize(img, [HEIGHT, WIDTH], 'nearest')
    msk = tf.image.resize(msk, [HEIGHT, WIDTH], 'nearest')
    
    msk = tf.cast(msk, tf.float32)
    combined = tf.concat([img, msk], axis=2)
    combined = tf.image.random_crop(combined, [IMG_HEIGHT, IMG_WIDTH, 4])
    img = combined[:, :, :3]
    msk = combined[:, :, -1]
    msk = tf.expand_dims(msk, axis = 2) 
    
#     img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH], 'nearest')
#     msk = tf.image.resize(msk, [IMG_HEIGHT, IMG_WIDTH], 'nearest')
    
    msk = tf.cast(msk, tf.int32)
    
    return img, msk

def random_flip_horz_vert(img, msk):
    if tf.random.uniform(()) > 0.5:
        img = tf.image.flip_left_right(img)
        msk = tf.image.flip_left_right(msk)
    if tf.random.uniform(()) > 0.5:
        img = tf.image.flip_up_down(img)
        msk = tf.image.flip_up_down(msk)
    
    return img, msk
  

def random_rot(img, msk):
    k = tf.random.uniform([], minval = 0, maxval = 4, dtype = tf.int32)
    img = tf.image.rot90(img,  k)
    msk = tf.image.rot90(msk, k)
    return img, msk
  
def preprocess(img_path, mask_path):
    img, msk = load_img(img_path, mask_path)
    img, msk = random_crop(img, msk)
    img, msk = random_flip_horz_vert(img, msk)
    img, msk = random_rot(img, msk)
    return img, msk

def preprocess2(img_path, mask_path):
    img, msk = load_img(img_path, mask_path)
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH], 'nearest')
    msk = tf.image.resize(msk, [IMG_HEIGHT, IMG_WIDTH], 'nearest')

    return img, msk
  



In [0]:
listA = tf.data.Dataset.list_files(DATA + 'img' +'/*.jpg', shuffle=False)
listB = tf.data.Dataset.list_files(DATA + 'mask' + '/*.png', shuffle=False)

listA = [path for path in iter(listA)]
listB = [path for path in iter(listB)]

train_listA = listA[:train_size]
train_listB = listB[:train_size]
test_listA = listA[train_size:]
test_listB = listB[train_size:]

# train data
train_data = tf.data.Dataset.from_tensor_slices((train_listA, train_listB))
train_data = train_data.shuffle(train_size)
train_data = train_data.map(preprocess)
train_data = train_data.batch(batch_size)
train_temp = train_data

# test data
test_data = tf.data.Dataset.from_tensor_slices((test_listA, test_listB))
test_data_temp = test_data.map(preprocess2).batch(batch_size)
test_data = test_data.map(preprocess2).repeat()
test_data = test_data.batch(batch_size)
test_temp = test_data

# print(train_data)
# print(next(iter(train_data))[0].shape)
# print(next(iter(train_data))[1].shape)

train_sample_img, train_sample_mask = next(iter(train_temp))
print('train_sample_img:', train_sample_img.shape, train_sample_img.dtype, train_sample_img.numpy().min(), train_sample_img.numpy().max())
print('train_sample_mask:', train_sample_mask.shape, train_sample_mask.dtype, train_sample_mask.numpy().min(), train_sample_mask.numpy().max())


test_sample_img, test_sample_mask = next(iter(test_temp))
print('test_sample_img:', test_sample_img.shape, test_sample_img.dtype, test_sample_img.numpy().min(), test_sample_img.numpy().max())
print('test_sample_mask:', test_sample_mask.shape, test_sample_mask.dtype, test_sample_mask.numpy().min(), test_sample_mask.numpy().max())


train_sample_img: (64, 256, 256, 3) <dtype: 'float32'> -1.0 1.0
train_sample_mask: (64, 256, 256, 1) <dtype: 'int32'> 0 3
test_sample_img: (64, 256, 256, 3) <dtype: 'float32'> -1.0 1.0
test_sample_mask: (64, 256, 256, 1) <dtype: 'int32'> 0 3


In [0]:
# i = 0
# plt.subplot(221)
# plt.title('image')
# plt.imshow(train_sample_img[i] * 0.5 + 0.5)

# train_sample_mask = tf.cast(train_sample_mask, tf.float32)
# test_sample_mask = tf.cast(test_sample_mask, tf.float32)

# plt.subplot(222)
# plt.title('mask')
# plt.imshow(tf.squeeze(train_sample_mask[i])/3.0, 'gray')

# plt.subplot(223)
# plt.title('image')
# plt.imshow(test_sample_img[i] * 0.5 + 0.5)

# plt.subplot(224)
# plt.title('mask')
# plt.imshow(tf.squeeze(test_sample_mask[i])/3.0, 'gray')

# print('img: ')
# print(tf.reduce_min(train_sample_img))
# print(tf.reduce_max(train_sample_img))

# print('msk:')
# print(tf.reduce_min(train_sample_mask))
# print(tf.reduce_max(train_sample_mask))

# print('img: ')
# print(tf.reduce_min(test_sample_img))
# print(tf.reduce_max(test_sample_img))

# print('msk:')
# print(tf.reduce_min(test_sample_mask))
# print(tf.reduce_max(test_sample_mask))

# Generator

In [0]:
# class SpectralNormalization(keras.layers.Wrapper):
#     """
#     Attributes:
#        layer: tensorflow keras layers (with kernel attribute)
#     """

#     def __init__(self, layer, **kwargs):
#         super(SpectralNormalization, self).__init__(layer, **kwargs)

#     def build(self, input_shape):
#         """Build `Layer`"""

#         if not self.layer.built:
#             self.layer.build(input_shape)

#             if not hasattr(self.layer, 'kernel'):
#                 raise ValueError(
#                     '`SpectralNormalization` must wrap a layer that'
#                     ' contains a `kernel` for weights')
#             self.w = self.layer.kernel
#             self.w_shape = self.w.shape.as_list()
#             self.u = self.add_weight(
#                 shape=tuple([1, self.w_shape[-1]]),
#                 initializer=tf.initializers.TruncatedNormal(stddev=0.02),
#                 name='sn_u',
#                 trainable=False,
#                 dtype=tf.float32)

#         super(SpectralNormalization, self).build()

#     @tf.function
#     def call(self, inputs):
#         """Call `Layer`"""
#         # Recompute weights for each forward pass
#         self._compute_weights()
#         output = self.layer(inputs)
#         return output

#     def _compute_weights(self):
#         """Generate normalized weights.
#         This method will update the value of self.layer.kernel with the
#         normalized value, so that the layer is ready for call().
#         """
#         w_reshaped = tf.reshape(self.w, [-1, self.w_shape[-1]])
#         eps = 1e-12
#         _u = tf.identity(self.u)
#         _v = tf.matmul(_u, tf.transpose(w_reshaped))
#         _v = _v / tf.maximum(tf.reduce_sum(_v**2)**0.5, eps)
#         _u = tf.matmul(_v, w_reshaped)
#         _u = _u / tf.maximum(tf.reduce_sum(_u**2)**0.5, eps)

#         self.u.assign(_u)
#         sigma = tf.matmul(tf.matmul(_v, w_reshaped), tf.transpose(_u))
        
#         self.layer.kernel.assign(self.w / sigma)

#     def compute_output_shape(self, input_shape):
#         return tf.TensorShape(
#             self.layer.compute_output_shape(input_shape).as_list())

In [0]:
class Downsample(keras.Model):

    def __init__(self, filters, size, apply_batchnorm=True):
        super(Downsample, self).__init__()
        
        self.apply_batchnorm = apply_batchnorm
        initializer = tf.random_normal_initializer(0., 0.02)

        self.conv1 = keras.layers.Conv2D(filters,
                                            (size, size),
                                            strides=2,
                                            padding='same',
                                            kernel_initializer=initializer,
                                            use_bias=False)
        if self.apply_batchnorm:
            self.batchnorm = keras.layers.BatchNormalization()

    def call(self, x, training):
        x = self.conv1(x)
        if self.apply_batchnorm:
            x = self.batchnorm(x, training=training)
        x = tf.nn.leaky_relu(x)
        return x


class Upsample(keras.Model):

    def __init__(self, filters, size, apply_dropout=False):
        super(Upsample, self).__init__()
        
        self.apply_dropout = apply_dropout
        initializer = tf.random_normal_initializer(0., 0.02)

        self.up_conv = keras.layers.Conv2DTranspose(filters,
                                                       (size, size),
                                                       strides=2,
                                                       padding='same',
                                                       kernel_initializer=initializer,
                                                       use_bias=False)
        self.batchnorm = keras.layers.BatchNormalization()
        if self.apply_dropout:
            self.dropout = keras.layers.Dropout(drop_rate)

    def call(self, x1, x2, training=None):

        x = self.up_conv(x1)
        x = self.batchnorm(x, training=training)
        if self.apply_dropout:
            x = self.dropout(x, training=training)
        x = tf.nn.relu(x)
        x = tf.concat([x, x2], axis=-1)
        return x
      
class GeneratorB2A(keras.Model):

    def __init__(self):
        super(GeneratorB2A, self).__init__()

        initializer = tf.random_normal_initializer(0., 0.02)

        self.down1 = Downsample(64, 4, apply_batchnorm=False)
        self.down2 = Downsample(128, 4)
        self.down3 = Downsample(256, 4)
        self.down4 = Downsample(512, 4)
        self.down5 = Downsample(512, 4)
        self.down6 = Downsample(512, 4)
        self.down7 = Downsample(512, 4)
        self.down8 = Downsample(512, 4)
        self.up1 = Upsample(512, 4, apply_dropout=True)
        self.up2 = Upsample(512, 4, apply_dropout=True)
        self.up3 = Upsample(512, 4, apply_dropout=True)
        self.up4 = Upsample(512, 4)
        self.up5 = Upsample(256, 4)
        self.up6 = Upsample(128, 4)
        self.up7 = Upsample(64, 4)

        self.last = keras.layers.Conv2DTranspose(3, (4, 4),
                                                    strides=2,
                                                    padding='same',
                                                    kernel_initializer=initializer)


    def call(self, inputs, training=None):
        # x shape == (bs, 256, 256, 4) 
        x, noise = inputs   
        x = tf.concat([x, noise], axis=-1)
        x1 = self.down1(x, training=training)  # (bs, 128, 128, 64)
        x2 = self.down2(x1, training=training)  # (bs, 64, 64, 128)
        x3 = self.down3(x2, training=training)  # (bs, 32, 32, 256)
        x4 = self.down4(x3, training=training)  # (bs, 16, 16, 512)
        x5 = self.down5(x4, training=training)  # (bs, 8, 8, 512)
        x6 = self.down6(x5, training=training)  # (bs, 4, 4, 512)
        x7 = self.down7(x6, training=training)  # (bs, 2, 2, 512)
        x8 = self.down8(x7, training=training)  # (bs, 1, 1, 512)

        x9 = self.up1(x8, x7, training=training)  # (bs, 2, 2, 1024)
        x10 = self.up2(x9, x6, training=training)  # (bs, 4, 4, 1024)
        x11 = self.up3(x10, x5, training=training)  # (bs, 8, 8, 1024)
        x12 = self.up4(x11, x4, training=training)  # (bs, 16, 16, 1024)
        x13 = self.up5(x12, x3, training=training)  # (bs, 32, 32, 512)
        x14 = self.up6(x13, x2, training=training)  # (bs, 64, 64, 256)
        x15 = self.up7(x14, x1, training=training)  # (bs, 128, 128, 128)

        x16 = self.last(x15)  # (bs, 256, 256, 3)
        x16 = tf.nn.tanh(x16)

        return x16

    def model(self):
        x = tf.keras.layers.Input(shape=[IMG_HEIGHT, IMG_HEIGHT, 4], name='input_image')
        seed = tf.keras.layers.Input(shape=[IMG_HEIGHT, IMG_HEIGHT, 1], name='seed')
        return keras.Model(inputs=[x, seed], outputs=self.call([x, seed]))




# Discriminator

In [0]:
class DiscDownsample(keras.Model):

    def __init__(self, filters, size, apply_batchnorm=True):
        super(DiscDownsample, self).__init__()

        self.apply_batchnorm = apply_batchnorm
        initializer = tf.random_normal_initializer(0., 0.02)

        self.conv1 = keras.layers.Conv2D(filters, (size, size),
                                            strides=2,
                                            padding='same',
                                            kernel_initializer=initializer,
                                            use_bias=False)
        if self.apply_batchnorm:
            self.batchnorm = keras.layers.BatchNormalization()

    def call(self, x, training=None):

        x = self.conv1(x)
        if self.apply_batchnorm:
            x = self.batchnorm(x, training=training)
        x = tf.nn.leaky_relu(x)
        return x
      
class DiscriminatorA(keras.Model):

    def __init__(self):
        super(DiscriminatorA, self).__init__()

        initializer = tf.random_normal_initializer(0., 0.02)

        self.down1 = DiscDownsample(64, 4, False)
        self.down2 = DiscDownsample(128, 4)
        self.down3 = DiscDownsample(256, 4)

        # we are zero padding here with 1 because we need our shape to 
        # go from (batch_size, 32, 32, 256) to (batch_size, 31, 31, 512)
        self.zero_pad1 = keras.layers.ZeroPadding2D()
        self.conv = keras.layers.Conv2D(512, (4, 4),
                                           strides=1,
                                           kernel_initializer=initializer,
                                           use_bias=False)
        self.batchnorm1 = keras.layers.BatchNormalization()

        # shape change from (batch_size, 31, 31, 512) to (batch_size, 30, 30, 1)
        self.zero_pad2 = keras.layers.ZeroPadding2D()
        self.last = keras.layers.Conv2D(1, (4, 4),
                                           strides=1,
                                           kernel_initializer=initializer)


    def call(self, inputs, training=None):
        inp, target = inputs

        # concatenating the input and the target
        x = tf.concat([inp, target], axis=-1)  # (bs, 256, 256, 7)
        x = self.down1(x, training=training)  # (bs, 128, 128, 64)
        x = self.down2(x, training=training)  # (bs, 64, 64, 128)
        x = self.down3(x, training=training)  # (bs, 32, 32, 256)

        x = self.zero_pad1(x)  # (bs, 34, 34, 256)
        x = self.conv(x)  # (bs, 31, 31, 512)
        x = self.batchnorm1(x, training=training)
        x = tf.nn.leaky_relu(x)

        x = self.zero_pad2(x)  # (bs, 33, 33, 512)
        # don't add a sigmoid activation here since
        # the loss function expects raw logits.
        x = self.last(x)  # (bs, 30, 30, 1)

        return x
    
    def model(self):
        inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
        tar = tf.keras.layers.Input(shape=[256, 256, 4], name='target_image')
        return keras.Model(inputs=[inp,  tar], outputs=self.call([inp,  tar]))

# test

In [0]:
generator1 = GeneratorB2A()
generator1.build(input_shape=[(batch_size, 256, 256, 4), (batch_size, 256, 256, 1)])
generator1.model()
generator1.summary()

discriminator = DiscriminatorA()
discriminator.build(input_shape=[(batch_size, 256, 256, 3), (batch_size, 256, 256, 4)])
discriminator.model()
discriminator.summary()


Model: "generator_b2a"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
downsample (Downsample)      (None, 128, 128, 64)      5120      
_________________________________________________________________
downsample_1 (Downsample)    (None, 64, 64, 128)       131584    
_________________________________________________________________
downsample_2 (Downsample)    (None, 32, 32, 256)       525312    
_________________________________________________________________
downsample_3 (Downsample)    (None, 16, 16, 512)       2099200   
_________________________________________________________________
downsample_4 (Downsample)    (None, 8, 8, 512)         4196352   
_________________________________________________________________
downsample_5 (Downsample)    (None, 4, 4, 512)         4196352   
_________________________________________________________________
downsample_6 (Downsample)    (None, 2, 2, 512)       

# Loss

In [0]:
binary_loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(disc_generated_output, gen_output, target, glamda=10):
    
    gan_loss = binary_loss_obj(tf.ones_like(disc_generated_output), disc_generated_output)
    
    l1_loss = tf.abs(gen_output - target)
    l1_loss = tf.reduce_mean(l1_loss)
    
    gan_loss = tf.reduce_mean(gan_loss)

    total_gen_loss = gan_loss + glamda * l1_loss

    return total_gen_loss
    # , gan_loss, l1_loss

# def gradient_penalty(discriminator, img, gen_img, label):
#     t = tf.random.uniform(shape=img.shape, minval=0, maxval=1)
#     interplate = t * img + (1 - t) * gen_img
#     with tf.GradientTape() as tape:
#         tape.watch([interplate])
#         mix_output = discriminator([interplate, label])
#     grad = tape.gradient(mix_output, interplate)
    
#     norm = tf.norm(tf.reshape(grad,[grad.shape[0], -1]), axis=1)
  
#     gp = tf.square(norm - 1.)
    
#     return gp 

# def discriminator_loss(discA_fake_output, genB2A_output, trainA, glambda=glambda)
def discriminator_loss(real_output, gen_output, dlamda=10):
  
    # real_output = discriminator([img, label], training=training)
    # gen_output = discriminator([gen_img, label], training=training)
    loss1 = binary_loss_obj(tf.ones_like(real_output), real_output)
    loss2 = binary_loss_obj(tf.zeros_like(gen_output), gen_output)
    # penalty = tf.reduce_mean(gradient_penalty(discriminator, img, gen_img, label))
    total = tf.reduce_mean(loss1) + tf.reduce_mean(loss2)
    # loss = total + dlamda * penalty
    
    return total
    # , penalty


# load model

In [0]:
discA = DiscriminatorA()
genB2A = GeneratorB2A()

genB2A_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discA_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)


# Checkpoint

In [0]:
!rm -rf ./ckpt/$version

In [0]:
!mkdir ./ckpt/$version

In [0]:
checkpoint_dir = './ckpt/' + version + '/'
checkpoint_prefix = os.path.join(checkpoint_dir, version + '_ckpt')

checkpoint = tf.train.Checkpoint( genB2A=genB2A,
                                  discA=discA,
                                  genB2A_optimizer=genB2A_optimizer,
                                  discA_optimizer=discA_optimizer)

print(checkpoint_dir)

./ckpt/mask2img/


# TOOL

## generate image

In [0]:
def convert_mask(mask):
  mask = tf.math.softmax(mask, axis = 3)
  mask = tf.argmax(mask, axis = 3)    
  mask = tf.cast(mask, tf.float32)
  
  return mask

def generate_images(genB2A, img, mask, path, epoch):
  z = tf.random.normal([img.shape[0], IMG_HEIGHT, IMG_WIDTH, 1])
  fake_img = genB2A([mask,z], training=True)
  mask = convert_mask(mask)
  
  for j in range(JJ):
    plt.figure(figsize=(15, 5))
    display_list = [img[j], mask[j], fake_img[j]]
    title = ['img', 'mask', 'fake_img']
    for i in range(3):
      plt.subplot(1, 3, i+1)
      plt.title(title[i])
      if i == 0 or i == 2:
        plt.imshow(display_list[i] * 0.5 + 0.5)
      else:
        plt.imshow(display_list[i]/3.0, 'gray')
      plt.axis('off')
    plt.savefig(path + '/{}_{:04d}.png'.format(j, epoch))
    plt.close()

  
def generate_all_images(genB2A, img, mask, path, begin):
  z = tf.random.normal([img.shape[0], IMG_HEIGHT, IMG_WIDTH, 1])
  fake_img = genB2A([mask, z], training=False)
  mask = convert_mask(mask)
  
  for j in range(img.shape[0]):
    plt.figure(figsize=(15, 5))
    display_list = [img[j], mask[j], fake_img[j]]
    title = ['img', 'mask', 'fake_img']
    for i in range(3):
      plt.subplot(1, 3, i+1)
      plt.title(title[i])
      if i == 0 or i == 2:
        plt.imshow(display_list[i] * 0.5 + 0.5)
      else:
        plt.imshow(display_list[i]/3.0, 'gray')
      plt.axis('off')
    plt.savefig(path + '/' + version +'_all_{:04d}.png'.format(begin + j))
    plt.close()

## GIF

In [0]:
# def make_gif():
#   paths = ['./train/' + version, './test/' + version]
#   ways = ['train', 'test']
#   for i in range(0, 2):
#     path = paths[i]
#     t = i
#     for j in range(JJ):
#       anim_file = version + '_{}_{}.gif'.format(ways[t], j)
#       with imageio.get_writer(anim_file, mode='I') as writer:
#         filenames = glob.glob(path +'/{}_*.png'.format(j))
#         filenames = sorted(filenames)
#         last = -1
#         for i,filename in enumerate(filenames):
#           frame = 2*(i**0.5)
#           if round(frame) > round(last):
#             last = frame
#           else:
#             continue
#           image = imageio.imread(filename)
#           writer.append_data(image)
#         image = imageio.imread(filename)
#         writer.append_data(image)
  

## FID

In [0]:
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1, steps=1)
	act2 = model.predict(images2, steps=1)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = np.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid
	
def calculate_fid_with_img(img1, img2):
 img1 = tf.image.resize(img1, [299, 299], 'nearest')
 img2 = tf.image.resize(img2, [299, 299], 'nearest')
 fid = calculate_fid(inception3, img1, img2)
 return fid


In [0]:
# fid_score = calculate_fid_with_img(train_sample_img, test_sample_img)
# print(fid_score)

# LOGS

In [0]:
 !rm -rf ./logs/$version

In [0]:
# current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = './logs/' + version
train_summary_writer = tf.summary.create_file_writer(log_dir)

# Train step

In [0]:
@tf.function
def train_step(trainA, trainB, glamda, dlamda):
  z = tf.random.normal([trainB.shape[0], IMG_HEIGHT, IMG_WIDTH, 1])
  with tf.GradientTape() as genB2A_tape, tf.GradientTape() as discA_tape:
      genB2A_output = genB2A([trainB, z], training=True)
      discA_real_output = discA([trainA, trainB], training=True)
      discA_fake_output = discA([genB2A_output, trainB], training=True)
      discA_loss = discriminator_loss(discA_real_output, discA_fake_output, dlamda)
      # discA_loss = discriminator_loss(discA, trainA, genB2A_output, trainB, training=True, dlamda=dlamda)
      genB2A_loss= generator_loss(discA_fake_output, genB2A_output, trainA, glamda)

  genB2A_gradients = genB2A_tape.gradient(genB2A_loss, genB2A.trainable_variables)
  discA_gradients = discA_tape.gradient(discA_loss, discA.trainable_variables)

  genB2A_optimizer.apply_gradients(zip(genB2A_gradients, genB2A.trainable_variables))
  discA_optimizer.apply_gradients(zip(discA_gradients, discA.trainable_variables))

  return genB2A_loss, discA_loss

# Train

In [0]:
!rm -rf ./train/$version
!rm -rf ./test/$version
!mkdir ./train/$version
!mkdir ./test/$version

In [0]:
def train():
  train_temp_ = iter(train_temp)
  test_temp_  = iter(test_temp)
  train_sample_img, train_sample_mask = next(train_temp_)
  test_sample_img, test_sample_mask = next(test_temp_)

  train_sample_mask = tf.one_hot(tf.squeeze(train_sample_mask, axis=-1), 4, on_value=1.0, off_value=0.0, axis=3)
  test_sample_mask = tf.one_hot(tf.squeeze(test_sample_mask, axis=-1), 4, on_value=1.0, off_value=0.0, axis=3)
  print('batch_size:', batch_size, ', epochs:', epochs, ', glamda:', glamda, ', dlamda:', dlamda, ', drop_rate:', drop_rate)

  print('Begin time:{}.'.format( time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())))
  start_time = time.time()
  input_image = 0
  for epoch in range(1, epochs + 1):
    for inputs in tf.data.Dataset.zip(train_data):
      input_image, target = inputs
      target = tf.one_hot(tf.squeeze(target, axis=-1), 4, on_value=1.0, off_value=0.0, axis=3)
      gloss, dloss = train_step(input_image, target, dlamda=dlamda, glamda=glamda)
    
    img, tar = next(test_temp_)
    tar = tf.one_hot(tf.squeeze(tar, axis=-1), 4, on_value=1.0, off_value=0.0, axis=3)
    seed_tr = tf.random.normal([target.shape[0], IMG_HEIGHT, IMG_WIDTH, 1])
    seed_te = tf.random.normal([tar.shape[0], IMG_HEIGHT, IMG_WIDTH, 1])
    train_fid = calculate_fid_with_img(input_image, genB2A([target, seed_tr], training=True))
    test_fid = calculate_fid_with_img(img, genB2A([tar, seed_te], training=True))
    if epoch % 50 == 0:
        generate_images(genB2A, train_sample_img, train_sample_mask, './train/' + version, epoch)
        generate_images(genB2A, test_sample_img, test_sample_mask, './test/' + version, epoch)
    
    with train_summary_writer.as_default():
      tf.summary.scalar('gloss',  float(gloss), step=epoch)
      # tf.summary.scalar('lloss',  float(lloss), step=epoch)
      tf.summary.scalar('dloss', float(dloss), step=epoch)
      # tf.summary.scalar('gp', float(gp), step=epoch)
      tf.summary.scalar('train_fid', train_fid, step=epoch)
      tf.summary.scalar('test_fid', test_fid, step=epoch)
    print('Epoch: {}, Current time:{}, gloss:{:.2f},dloss:{:.2f},train_fid:{:.2f},test_fid:{:.2f}'.format(epoch, time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), float(gloss),float(dloss),train_fid, test_fid))
    # print('Epoch: {}, Current time:{}, gloss:{:.2f}, lloss:{:.2f} dloss:{:.2f}, gp:{:.2f}, train_fid:{:.2f}, test_fid:{:.2f}'.format(epoch, time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), float(gloss), float(lloss),float(dloss), float(gp), train_fid, test_fid))

    if epoch + 10 > epochs:
      checkpoint.save(file_prefix = checkpoint_prefix)
      print('Epoch: {}, Current time:{}, save checkpoint successfully. '.format(epoch, time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())))

  checkpoint.save(file_prefix = checkpoint_prefix)
  end_time = time.time()
  total_sec = end_time - start_time
  hours = int((total_sec)/(60*60))
  minitues = int((total_sec/60 - hours*60))
  seconds = int(total_sec - hours*3600 - minitues*60)
  print('End time:{}, total use time:{}hours, {}minitues, {}seconds.'.format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), hours, minitues, seconds))


# Test

In [0]:
def test():
  begin = 0
  for inputs in tf.data.Dataset.zip(test_data_temp):
    input_image, target = inputs
    target = tf.one_hot(tf.squeeze(target, axis=-1), 4, on_value=1.0, off_value=0.0, axis=3)
    generate_all_images(genB2A, input_image, target, './test/' + version, begin)
    begin += batch_size
    print(begin)

# MAIN

In [0]:
# !kill 419
%tensorboard --logdir ./logs

In [0]:
if __name__ == '__main__':
#   checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
  print('\n-------------------------------------------TRAIN-------------------------------------------\n')
  train()
  print('\n-------------------------------------------TEST-------------------------------------------\n')
  test()
  # print('\n-------------------------------------------GIF-------------------------------------------\n')
  # make_gif()


-------------------------------------------TRAIN-------------------------------------------

batch_size: 64 , epochs: 1500 , glamda: 3 , dlamda: 0 , drop_rate: 0.3
Begin time:2019-11-16 07:44:53.
Epoch: 1, Current time:2019-11-16 07:47:14, gloss:2.19,dloss:1.41,train_fid:503.90,test_fid:508.47
Epoch: 2, Current time:2019-11-16 07:47:52, gloss:2.03,dloss:1.23,train_fid:498.55,test_fid:476.64
Epoch: 3, Current time:2019-11-16 07:48:30, gloss:2.14,dloss:1.49,train_fid:527.97,test_fid:516.05
Epoch: 4, Current time:2019-11-16 07:49:09, gloss:1.82,dloss:1.34,train_fid:485.53,test_fid:452.57
Epoch: 5, Current time:2019-11-16 07:49:43, gloss:1.65,dloss:1.23,train_fid:468.28,test_fid:467.74
Epoch: 6, Current time:2019-11-16 07:50:09, gloss:1.71,dloss:1.26,train_fid:431.61,test_fid:420.93
Epoch: 7, Current time:2019-11-16 07:50:35, gloss:1.76,dloss:1.38,train_fid:435.76,test_fid:427.75
Epoch: 8, Current time:2019-11-16 07:51:01, gloss:1.80,dloss:1.38,train_fid:413.72,test_fid:367.80
Epoch: 9, C

In [0]:
def train():
  train_temp_ = iter(train_temp)
  test_temp_  = iter(test_temp)
  train_sample_img, train_sample_mask = next(train_temp_)
  test_sample_img, test_sample_mask = next(test_temp_)

  train_sample_mask = tf.one_hot(tf.squeeze(train_sample_mask, axis=-1), 4, on_value=1.0, off_value=0.0, axis=3)
  test_sample_mask = tf.one_hot(tf.squeeze(test_sample_mask, axis=-1), 4, on_value=1.0, off_value=0.0, axis=3)
  print('batch_size:', batch_size, ', epochs:', epochs, ', glamda:', glamda, ', dlamda:', dlamda, ', drop_rate:', drop_rate)

  print('Begin time:{}.'.format( time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())))
  start_time = time.time()
  input_image = 0
  for epoch in range(start + 1, epochs + 1):
    for inputs in tf.data.Dataset.zip(train_data):
      input_image, target = inputs
      target = tf.one_hot(tf.squeeze(target, axis=-1), 4, on_value=1.0, off_value=0.0, axis=3)
      gloss, dloss = train_step(input_image, target, dlamda=dlamda, glamda=glamda)
    
    img, tar = next(test_temp_)
    tar = tf.one_hot(tf.squeeze(tar, axis=-1), 4, on_value=1.0, off_value=0.0, axis=3)
    seed_tr = tf.random.normal([target.shape[0], IMG_HEIGHT, IMG_WIDTH, 1])
    seed_te = tf.random.normal([tar.shape[0], IMG_HEIGHT, IMG_WIDTH, 1])
    train_fid = calculate_fid_with_img(input_image, genB2A([target, seed_tr], training=True))
    test_fid = calculate_fid_with_img(img, genB2A([tar, seed_te], training=True))
    if epoch % 50 == 0:
        generate_images(genB2A, train_sample_img, train_sample_mask, './train/' + version, epoch)
        generate_images(genB2A, test_sample_img, test_sample_mask, './test/' + version, epoch)
    
    with train_summary_writer.as_default():
      tf.summary.scalar('gloss',  float(gloss), step=epoch)
      # tf.summary.scalar('lloss',  float(lloss), step=epoch)
      tf.summary.scalar('dloss', float(dloss), step=epoch)
      # tf.summary.scalar('gp', float(gp), step=epoch)
      tf.summary.scalar('train_fid', train_fid, step=epoch)
      tf.summary.scalar('test_fid', test_fid, step=epoch)
    if epoch % 10 == 0:
      print('Epoch: {}, Current time:{}, gloss:{:.2f},dloss:{:.2f},train_fid:{:.2f},test_fid:{:.2f}'.format(epoch, time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), float(gloss),float(dloss),train_fid, test_fid))
    # print('Epoch: {}, Current time:{}, gloss:{:.2f}, lloss:{:.2f} dloss:{:.2f}, gp:{:.2f}, train_fid:{:.2f}, test_fid:{:.2f}'.format(epoch, time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), float(gloss), float(lloss),float(dloss), float(gp), train_fid, test_fid))

    if epoch + 10 > epochs:
      checkpoint.save(file_prefix = checkpoint_prefix)
      print('Epoch: {}, Current time:{}, save checkpoint successfully. '.format(epoch, time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())))

  checkpoint.save(file_prefix = checkpoint_prefix)
  end_time = time.time()
  total_sec = end_time - start_time
  hours = int((total_sec)/(60*60))
  minitues = int((total_sec/60 - hours*60))
  seconds = int(total_sec - hours*3600 - minitues*60)
  print('End time:{}, total use time:{}hours, {}minitues, {}seconds.'.format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), hours, minitues, seconds))


In [0]:
start = 1500
epochs = 3000

In [0]:
%tensorboard --logdir ./logs

In [0]:
if __name__ == '__main__':
  checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
  print('\n-------------------------------------------TRAIN-------------------------------------------\n')
  train()
  print('\n-------------------------------------------TEST-------------------------------------------\n')
  test()
  # print('\n-------------------------------------------GIF-------------------------------------------\n')
  # make_gif()


-------------------------------------------TRAIN-------------------------------------------

batch_size: 64 , epochs: 3000 , glamda: 3 , dlamda: 0 , drop_rate: 0.3
Begin time:2019-11-16 19:59:09.
Epoch: 1510, Current time:2019-11-16 20:07:23, gloss:1.64,dloss:1.33,train_fid:140.97,test_fid:154.39
Epoch: 1520, Current time:2019-11-16 20:12:03, gloss:1.63,dloss:1.39,train_fid:148.40,test_fid:168.97
Epoch: 1530, Current time:2019-11-16 20:16:42, gloss:1.66,dloss:1.34,train_fid:157.30,test_fid:192.93
Epoch: 1540, Current time:2019-11-16 20:21:24, gloss:1.71,dloss:1.35,train_fid:151.13,test_fid:197.23
Epoch: 1550, Current time:2019-11-16 20:26:04, gloss:1.64,dloss:1.31,train_fid:145.18,test_fid:177.26
Epoch: 1560, Current time:2019-11-16 20:30:40, gloss:1.63,dloss:1.33,train_fid:152.59,test_fid:168.89
Epoch: 1570, Current time:2019-11-16 20:35:17, gloss:1.68,dloss:1.33,train_fid:152.75,test_fid:170.84
Epoch: 1580, Current time:2019-11-16 20:39:53, gloss:1.64,dloss:1.34,train_fid:133.16,tes